## Example for implementation of Spectrum estimation based on the Order Tracking

My objective is to increase the coverage for machines with unstable operational conditions (load and torque). I have bumped into a crane with an unstable operational speed. For this case, I want to use Order Tracking, allowing further utilization of the already implemented spectrum analysis tools. Later, I think that we can use this technique for other equipment too. 

In [ ]:
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
# %matplotlib notebook
figsize=[19, 5]
from scipy.signal import ellip, sosfreqz, sosfiltfilt, periodogram
from orderTrack import orderTrack, getSpeed

The magnetic file names

In [ ]:
files = glob('data/*/*magnetic*')
files

In [ ]:
def plotFiltResponse(sos, fs):
    w, h = sosfreqz(sos, fs=fs, worN=200000)
    dispCond = w < 20
    plt.figure(figsize=figsize)
    plt.plot(w[dispCond], abs(h[dispCond]))
    plt.title('Filter frequency response')
    plt.xlabel('Frequency [radians / second]')
    plt.ylabel('Amplitude')
    plt.grid()
    plt.show()

Magnetic file where the engine is on

In [ ]:
import os
filesMagnetic = []
for file in files:
    x = np.load(file)
    if 'sos' not in locals():
        fs = 1 / x[0, 1]
        sosHigh = ellip(8, 1, 40, 2, btype='high', output='sos', fs=fs)
#         plotFiltResponse(sos, fs)
    temp = sosfiltfilt(sosHigh, x[1, :])  
    # looking for a file with activity
    cond1 = (np.percentile(temp, 80) - np.percentile(temp, 20)) > 0.5
    cond2 = x[1, :].mean() < 100
    if (cond1 & cond2):
        filesMagnetic.append(file)
    else:
        for file in glob(file.split('_plane_')[0] + '*'):
            os.remove(file) 
len(filesMagnetic)

Packing the data of the vibration signals with magnetic signal

In [ ]:
for fileMagnetic in filesMagnetic:
    print(fileMagnetic)
    filesVib = glob(fileMagnetic.split('_plane_')[0] + '*vibration*')
    vibData = [np.atleast_2d(np.load(file)[1, :]) for file in filesVib]
    if np.all(np.diff([d.shape[1] for d in vibData]) == 0):
        xVibration = np.concatenate(vibData).T
    else:
        print("size differences in the vibration files of " + fileMagnetic)
        continue
    xVibration -= xVibration.mean(axis=0)
    tVibration = np.load(filesVib[0])[0, :]
    
    x = np.load(fileMagnetic)
    tMagnetic = x[0, :]
    xMagnetic = x[1, :]
    fs = 1 / tMagnetic[1]
    
    plt.figure(figsize=figsize)
    plt.plot(tMagnetic, xMagnetic)
    plt.xlabel('Time [sec]')
    plt.ylabel('Magnetic flux')
    plt.grid()
    plt.show()
    
    speed, speedTime = getSpeed(xMagnetic, fs, isPlot=True, figsize=figsize)
    if speed is None:
        continue
    resampledMagnetic, resampledPhase, _, _, _, _ = orderTrack(xMagnetic, tMagnetic, speed, speedTime)
    
    plt.figure(figsize=figsize)
    
    resampFs = 1 / (resampledPhase[1] - resampledPhase[0])
    f, Pxx_den = periodogram(resampledMagnetic, resampFs, window='hanning')
    plt.semilogy(f, Pxx_den)
    f, Pxx_den = periodogram(xMagnetic, fs / speed.mean(), window='hanning')
    plt.semilogy(f, Pxx_den)
    plt.legend(['Resampled signal', 'Original signal'])
    plt.xlabel('Shaft Speed Order [-] / Frequency normalized by mean speed')
    plt.ylabel('PSD [V**2]')
    plt.xlim([0, 10])
    plt.xticks(np.arange(11))
    plt.grid()
    plt.show()
    
    resampledVibration, resampledPhase, _, _, _, _ = orderTrack(xVibration, tVibration, speed, speedTime)
    fs = 1 / tVibration[1]
    resampFs = 1 / (resampledPhase[1] - resampledPhase[0])
    
    for iSig in np.arange(resampledVibration.shape[1]):
        plt.figure(figsize=figsize)
        
        f, Pxx_den = periodogram(resampledVibration[:, iSig], resampFs, window='hanning')
        plt.plot(f, Pxx_den)
        f, Pxx_den = periodogram(xVibration[:, iSig], fs / speed.mean(), window='hanning')
        plt.plot(f, Pxx_den)
        plt.legend(['Resampled signal', 'Original signal'])
        plt.xlabel('Shaft Speed Order [-] / Frequency normalized by mean speed')
        plt.ylabel('PSD [V**2]')
        plt.xlim([0, 30])
        plt.xticks(np.arange(30))
        plt.grid()
        plt.show()

### cyclo-non-stationary analysis
Following presented an attempt to integrate the cyclostationarity with the angular resampling inspired by 
Abboud, Dany, et al. "The spectral analysis of cyclo-non-stationary signals." Mechanical Systems and Signal Processing (2016). It is yet not finished by it shows some potential...

In [ ]:
from scipy.signal import spectrogram
from scipy.fft import fftfreq, fft

def getThresh(x, nSTD=5):
    return np.std(x) * nSTD + np.median(x)

def getEESpec(CS):
    CS[CS < getThresh(CS)] = 0
    return CS.sum(axis=0)

def getCCoh(x, fs, norm=True, speed=np.nan, speedTime=np.nan):
    f, t, Sxx = spectrogram(x,
                            fs=fs, window='hann', 
                            nperseg=64, noverlap=32, nfft=256, 
                            detrend='constant', return_onesided=True, 
                            scaling='density', mode='psd')
    
    if not np.isnan(speed).all():
        Sxx, t, _, _, _, _ = orderTrack(Sxx, t, speed, speedTime)
        Sxx = Sxx.T
    
    fMod = fftfreq(t.shape[0], d=t[1] - t[0])
    SxxDetrend = Sxx - np.atleast_2d(Sxx.mean(axis=1)).T
    CS = np.abs(fft(SxxDetrend, t.shape[0], norm='forward'))
    if norm:
        CS = np.abs(CS / np.atleast_2d(np.median(CS, axis=1)).T)
    CS[f < 1000, :] = 0
    return fMod, f, CS

In [ ]:
fs = 1 / tVibration[1]
for a in range(3):
    fMod, f, CCoh = getCCoh(xVibration[:, a], fs)

    cond = fMod > 0
    vminVal = np.min(CCoh)
    vmaxVal = np.percentile(CCoh, 99.95)
    fMod = fMod[cond]
    CCoh = CCoh[:, cond]
    
    plt.figure(figsize=figsize)
    plt.pcolormesh(fMod, f, CCoh, vmin = vminVal, vmax = vmaxVal)
    plt.ylabel('Carrier Frequency [Hz]')
    plt.xlabel(r'Modulation Frequency [Hz]')
    plt.show()
    
    fMod2, f2, CCoh2 = getCCoh(xVibration[:, a], fs, True, speed, speedTime)

    cond = fMod2 > 0
    vminVal2 = np.min(CCoh2)
    vmaxVal2 = np.percentile(CCoh2, 99.95)
    fMod2 = fMod2[cond]
    CCoh2 = CCoh2[:, cond]
    
    plt.figure(figsize=figsize)
    plt.pcolormesh(fMod2, f2, CCoh2, vmin = vminVal2, vmax = vmaxVal2)
    plt.ylabel('Carrier Frequency [Hz]')
    plt.xlabel(r'Modulation Shaft Speed Order [Hz]')
    plt.show()
    
    plt.figure(figsize=figsize)
    plt.plot(fMod2, getEESpec(CCoh2), linewidth=3)
    plt.plot(fMod / speed.mean(), getEESpec(CCoh), linewidth=3)
    plt.legend(['Resampled', 'Original'])
    plt.ylabel('Enhanced Envelope Spectrum')
    plt.xlabel(r'Modulation Shaft Speed Order [Hz]|Modulation Frequency [Hz]')
    plt.xticks(np.arange(np.floor(fMod2[-1])))
    plt.xlim([fMod2[0], fMod2[-1]])
    plt.grid()
    plt.show()

For non-stationary cases, we can see an improvement in the spectrum elements' decomposition. Besides that, we can see the following improvements:
1. Reduction in the dc noise (probably from the hardware). Why?
2. Reduction in the leakage (this one is yet pending improvement). Have been seen in multiple locations (see the last magnetic signal spectrum). This probably happens due to the trimming of the signal at the precise multiplication of the cycle.
3. Concentration of the energy in the integration points, located at the harmonics of the shaft speed order.

Points to improve:
1. Trimming of the signal where the machine is off.
2. figure out why we have DC.